In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install scanpy --quiet

     |████████████████████████████████| 2.0 MB 5.1 MB/s 
     |████████████████████████████████| 9.4 MB 103.4 MB/s 
     |████████████████████████████████| 96 kB 6.6 MB/s 
     |████████████████████████████████| 88 kB 438 kB/s 
     |████████████████████████████████| 295 kB 76.9 MB/s 
     |████████████████████████████████| 965 kB 75.8 MB/s 
     |████████████████████████████████| 1.1 MB 77.9 MB/s 
     |████████████████████████████████| 63 kB 2.6 MB/s 


In [1]:
import torch
from torch import nn
from torch.autograd import Variable
import anndata as ad
import numpy as np
import os
import collections 
from argparse import Namespace
from torch.utils.data import Dataset, DataLoader
config = Namespace(
    DEVICE = 'cuda',
    BATCH_SIZE = 64,
    NUM_WORKERS = 4,
    N_GENES = 13431,
    N_PEAKS = 116465,
    N_CHANNELS = 64,
    MAX_SEQ_LEN_GEX = 1500,
    MAX_SEQ_LEN_ATAC = 15000,
    LEARNING_RATE = 0.0005
)

In [2]:
# adata_gex = ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/GEX_processed.h5ad")

In [3]:
# adata_atac = ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/ATAC_processed.h5ad")

In [4]:
def get_chr_index(adata_atac):
  r"""
  Output row indices for each chromosome for each chromosome
  Parameters
  ----------
  adata_atac
      annData for ATAC
  Returns
  -------
  chr_index
      Dictionary of indices for each chromosome
  """
  row_name = adata_atac.var.index
  chr_name = [c.split("-")[0] for c in row_name]
  lst = np.unique(chr_name) # names for chromosome

  chr_index = dict()
  for i in range(len(lst)):
    index = [a for a, l in enumerate(chr_name) if l == lst[i]]
    if lst[i] not in chr_index:
      chr_index[lst[i]]=index

  return chr_index

In [5]:
## Write cnn modules for gex modalities
class gexCNN(nn.Module):
    """customized  module"""
    #argument index is the poisition for each choromosome
    def __init__(self, kernel_size):
        super(gexCNN, self).__init__()

        # Conv layer
        self.in_channels = 1 
        self.out_channels = config.N_CHANNELS
        self.kernel_size = kernel_size   
        self.stride = 50 # TO CHANGE 
        self.padding = 25 # TO CHANGE
        self.pool_size = 2
        self.pool_stride = 1
        self.convs = nn.Sequential(
            nn.Conv1d(in_channels = self.in_channels, 
                      out_channels = self.out_channels, 
                      kernel_size = self.kernel_size,
                      stride = self.stride,
                      padding = self.padding),
            nn.LeakyReLU(),
            nn.MaxPool1d(kernel_size = self.pool_size,
                         stride = self.pool_stride)
        )

        # # FC layer
        # self.conv_out_features = int((config.N_GENES + 2*self.padding - self.kernel_size) / self.stride + 1)
        # self.fc_in_features = int((self.conv_out_features - self.pool_size) / self.pool_stride + 1) * self.out_channels
        # self.fc_out_feature = 300
        # self.fc = nn.Linear(in_features = self.fc_in_features, out_features = self.fc_out_feature) 

    def forward(self, x):
        r"""  
        Generate GEX embeddings
        
        Parameters
        ----------
        x
            Pre-processed GEX data (batch_size x 1 x N_GENES)
        
        Returns
        -------
        gex_embed
            GEX embeddings of a batch (batch_size x seq_len x dim_size)
        """
        gex_embed = self.convs(x.float())
        # gex_embed = torch.flatten(gex_embed, 1)
        # gex_embed = self.fc(gex_embed)
        return gex_embed.transpose(1,2).to(config.DEVICE)

In [6]:
# # Test for gexCNN()
# x = torch.tensor(np.asarray(csr_gex[:5].todense())).unsqueeze(1) # 5 cells
# print(x.size())
# model = gexCNN(kernel_size = 100)
# print(model(x).size())

In [7]:
# Write cnn modules for atac modalities
class atacCNN(nn.Module):
    #argument index is the poisition for each choromosome
    def __init__(self, index, kernel_size_1, kernel_size_2):
        super(atacCNN, self).__init__()
        self.index = index
        
        # Conv layer
        self.in_channels_1 = 1 
        self.out_channels_1 = int(config.N_CHANNELS / 2)
        self.kernel_size_1 = kernel_size_1
        self.stride_1 = 10 # TO CHANGE 
        self.padding_1 = 5 # TO CHANGE

        self.in_channels_2 = int(config.N_CHANNELS / 2)
        self.out_channels_2 = config.N_CHANNELS 
        self.kernel_size_2 = kernel_size_2
        self.stride_2 = 5 # TO CHANGE 
        self.padding_2 = 3 # TO CHANGE

        self.convs = nn.Sequential(
            nn.Conv1d(in_channels = self.in_channels_1, 
                      out_channels = self.out_channels_1, 
                      kernel_size = self.kernel_size_1,
                      stride = self.stride_1,
                      padding = self.padding_1),
            nn.LeakyReLU(),
            nn.MaxPool1d(kernel_size = 5, stride = 2),

            nn.Conv1d(in_channels = self.in_channels_2, 
                      out_channels = self.out_channels_2, 
                      kernel_size = self.kernel_size_2,
                      stride = self.stride_2,
                      padding = self.padding_2),
            nn.LeakyReLU(),
            nn.MaxPool1d(kernel_size = 2, stride = 1)             
        )



    def forward(self, x):
        r"""  
        Generate ATAC embeddings
        
        Parameters
        ----------
        x
            Pre-processed ATAC data (batch_size x 1 x N_PEAKS)
        
        Returns
        -------
        atac_embed
            ATAC embeddings of a batch (batch_size x seq_len x dim_size)
        """
        atac_embed = []
        for chr in self.index.keys(): 
            idx = self.index[chr]
            x_chr = x[:,:,idx]
            x_chr = self.convs(x_chr.float())
            atac_embed.append(x_chr)
        atac_embed = torch.cat(atac_embed, dim = 2)
        return atac_embed.transpose(1,2).to(config.DEVICE)

In [8]:
# # Test for ATAC_CNN()
# x = torch.tensor(np.asarray(csr_atac[:5].todense())).unsqueeze(1) # 5 cells
# print(x.size())
# # index = get_chr_index(adata_atac)
# model = atacCNN(kernel_size_1 = 50, kernel_size_2 = 10, index = index)
# print(model(x).size())

In [48]:
class MultimodalAttention(nn.Module):
    def __init__(self):
        super(MultimodalAttention, self).__init__()
        self.nhead_gex = 1
        self.nhead_atac = 4
        self.nhead_multi = 4
        self.nlayer_gex = 1
        self.nlayer_atac = 1
        self.nlayer_multi = 1

        self.encoder_layer_gex = nn.TransformerEncoderLayer(d_model = config.N_CHANNELS, nhead = self.nhead_gex)
        self.transformer_encoder_gex = nn.TransformerEncoder(self.encoder_layer_gex, num_layers = self.nlayer_gex)
        # self.linear_gex_0 = nn.LazyLinear(out_features = 1)
        self.linear1_gex_0 = nn.LazyLinear(out_features = 30)
        self.linear2_gex_0 = nn.LazyLinear(out_features = 300)


        self.encoder_layer_atac = nn.TransformerEncoderLayer(d_model = config.N_CHANNELS, nhead = self.nhead_atac)
        self.transformer_encoder_atac = nn.TransformerEncoder(self.encoder_layer_atac, num_layers = self.nlayer_atac)
        # self.linear_atac_0 = nn.LazyLinear(out_features = 1)
        self.linear1_atac_0 = nn.LazyLinear(out_features = 30)
        self.linear2_atac_0 = nn.LazyLinear(out_features = 300)

        self.encoder_layer_multi = nn.TransformerEncoderLayer(d_model = config.N_CHANNELS, nhead = self.nhead_multi)
        self.transformer_encoder_multi = nn.TransformerEncoder(self.encoder_layer_multi, num_layers = self.nlayer_multi)
        # self.linear_gex_1 = nn.LazyLinear(out_features = 1)
        # self.linear_atac_1 = nn.LazyLinear(out_features = 1)
        self.linear1_gex_1 = nn.LazyLinear(out_features = 30)
        self.linear2_gex_1 = nn.LazyLinear(out_features = 300)
        self.linear1_atac_1 = nn.LazyLinear(out_features = 30)
        self.linear2_atac_1 = nn.LazyLinear(out_features = 300)
    

    def forward(self, gex_embed, atac_embed):
      r"""  
      Incorporate two self-attention and one cross-attention module

      Parameters
      ----------
      gex_embed
          GEX embeddings of a batch (batch_size x seq_len_gex x dim_size)
      atac_embed
          ATAC embeddings of a batch (batch_size x seq_len_atac x dim_size)

      Returns
      -------
      ## TO FILL
      """
      seq_len_gex = gex_embed.size()[1]
      seq_len_atac = atac_embed.size()[1]

      gex_context = self.transformer_encoder_gex(gex_embed)
      atac_context = self.transformer_encoder_atac(atac_embed)

      # Average self-attention fragment representation
      # gex_out_0 = self.linear_gex_0(gex_context.permute(0,2,1)).squeeze(2)
      # atac_out_0 = self.linear_atac_0(atac_context.permute(0,2,1)).squeeze(2)
      gex_out_0  = self.linear1_gex_0(gex_context.permute(0,2,1)).squeeze(2)
      # print(gex_out_0.size())
      gex_out_0  = self.linear2_gex_0(gex_out_0.flatten(start_dim=1))
      # print(gex_out_0.size())
      atac_out_0 = self.linear1_atac_0(atac_context.permute(0,2,1)).squeeze(2)
      # print(atac_out_0.size())
      atac_out_0 = self.linear2_atac_0(atac_out_0.flatten(start_dim=1))
      # print(atac_out_0.size())

      multi_embed = torch.cat((gex_context, atac_context), dim = 1)
      multi_context = self.transformer_encoder_multi(multi_embed)
      
      multi_context_gex = multi_context[:, :seq_len_gex, :]
      multi_context_atac = multi_context[:, seq_len_gex:, :]
      
      # # Average cross-attention fragment representation
      # gex_out_1 = multi_context_gex.mean(dim = 1)
      # atac_out_1 = multi_context_atac.mean(dim = 1)
      gex_out_1  = self.linear1_gex_1(multi_context_gex.permute(0,2,1)).squeeze(2)
      gex_out_1  = self.linear2_gex_1(gex_out_1.flatten(start_dim=1))
      atac_out_1 = self.linear1_atac_1(multi_context_atac.permute(0,2,1)).squeeze(2)
      atac_out_1 = self.linear2_atac_1(atac_out_1.flatten(start_dim=1))

      return gex_out_0.to(config.DEVICE), gex_out_1.to(config.DEVICE), atac_out_0.to(config.DEVICE), atac_out_1.to(config.DEVICE)

In [47]:
# # index = get_chr_index(adata_atac)

# x_gex = torch.tensor(np.asarray(csr_gex[:5].todense())).unsqueeze(1).to(config.DEVICE) # 5 cells
# x_atac = torch.tensor(np.asarray(csr_atac[:5].todense())).unsqueeze(1).to(config.DEVICE) # 5 cells

# gex_cnn = gexCNN(kernel_size = 100).to(config.DEVICE)
# atac_cnn = atacCNN(kernel_size_1 = 50, kernel_size_2 = 10, index = index).to(config.DEVICE)
# multi_attention = MultimodalAttention().to(config.DEVICE)

# gex_embed = gex_cnn(x_gex).to(config.DEVICE)
# atac_embed = atac_cnn(x_atac).to(config.DEVICE)
# print(gex_embed.size()); print(atac_embed.size())
# gex_out_0, gex_out_1, atac_out_0, atac_out_1 = multi_attention(gex_embed, atac_embed)
# print(atac_out_0.size())

torch.Size([5, 267, 64])
torch.Size([5, 1122, 64])
torch.Size([5, 64, 30])
torch.Size([5, 300])
torch.Size([5, 64, 30])
torch.Size([5, 300])
torch.Size([5, 300])


In [29]:
# embedding = nn.Embedding(1000, 128)
# anchor_ids = torch.randint(0, 1000, (1,))
# positive_ids = torch.randint(0, 1000, (1,))
# negative_ids = torch.randint(0, 1000, (1,))
# anchor = embedding(anchor_ids)
# positive = embedding(positive_ids)
# negative = embedding(negative_ids)

In [30]:
# print(anchor.size())
# print(positive.size())
# print(negative.size())

In [49]:
class Encoder(nn.Module):
    def __init__(self, kernel_size_gex, kernel_size_atac_1, kernel_size_atac_2, index):
        super(Encoder, self).__init__()

        self.kernel_size_gex = kernel_size_gex
        self.kernel_size_atac_1 = kernel_size_atac_1
        self.kernel_size_atac_2 = kernel_size_atac_2
        self.index = index

        self.gex_cnn = gexCNN(kernel_size = self.kernel_size_gex)
        self.atac_cnn = atacCNN(kernel_size_1 = self.kernel_size_atac_1, kernel_size_2 = self.kernel_size_atac_2, index = self.index)
        self.multi_attention = MultimodalAttention()

        
    def forward(self, x_gex, x_atac):

        gex_embed = self.gex_cnn(x_gex)
        atac_embed = self.atac_cnn(x_atac)
        gex_out_0, gex_out_1, atac_out_0, atac_out_1 = self.multi_attention(gex_embed, atac_embed)

        return gex_out_0, gex_out_1, atac_out_0, atac_out_1


In [32]:
# gex_test = torch.tensor(np.asarray(adata_gex.layers['log_norm'][:5].todense())).unsqueeze(1) # 5 cells
# atac_test = torch.tensor(np.asarray(adata_atac.layers['log_norm'][:5].todense())).unsqueeze(1) # 5 cells

# index = get_chr_index(adata_atac)


In [33]:
# encoder = Encoder(kernel_size = 32, index = index )
# gex_out_0, gex_out_1, atac_out_0, atac_out_1 = encoder(gex_test, atac_test)

In [50]:
from numpy.lib.shape_base import row_stack
class bidirectTripletLoss(nn.Module):
    r"""
    
    Output bidirectional triplet loss for two pairs of gex and atac
    ----------
    gex_0_mat: Matrix of GEX embeddings from self-attention (batch_size x embedding_size_0)

    Returns
    -------
    loss
    """
    def __init__(self, alpha, margin):
        super(bidirectTripletLoss, self).__init__()

        self.alpha = alpha
        self.margin = margin
        self.cross_entropy_loss = nn.CrossEntropyLoss()

    def similarityScore(self, gex_out_0, gex_out_1, atac_out_0, atac_out_1):
        r"""
        Output similarity scores for two pairs of gex and atac
        ----------


        Returns
        -------
        score: batch_size * batch_size
        similarity score between two modalities
        """ 

        # print(gex_mat.size())
        # print(atac_mat.size())

        # gex_mat0, gex_mat1 = torch.split(gex_mat, config.N_CHANNELS, dim = 1)
        # atac_mat0, atac_mat1 = torch.split(atac_mat,config.N_CHANNELS, dim = 1)

        gex_out_0 = nn.functional.normalize(gex_out_0, dim = 1)
        gex_out_1 = nn.functional.normalize(gex_out_1, dim = 1)
        atac_out_0 = nn.functional.normalize(atac_out_0, dim = 1)
        atac_out_1 = nn.functional.normalize(atac_out_1, dim = 1)
        
        score = torch.mm(gex_out_0, atac_out_0.transpose(0,1)) + self.alpha * torch.mm(gex_out_1, atac_out_1.transpose(0,1))
        return  score.to(config.DEVICE)

    def triplet(self, score_mat):

        true_score = torch.diagonal(score_mat)
        # print("true_score:\n", true_score)
        # print("true_score dimension:\n", true_score.size())
        
        reduced_score_mat = score_mat - torch.diag(true_score) # set the diagnoal score to be zero
        
        neg_index_1 = torch.argmax(reduced_score_mat, dim = 1) # indices of hard negatives for GEX
        neg_index_2 = torch.argmax(reduced_score_mat, dim = 0) # indices of hard negatives for ATAC
        # print("neg_index_1:\n", neg_index_1); print("neg_index_2:\n", neg_index_2)
        # print("neg_index_1 dimension:\n", neg_index_1.size()); print("neg_index_2 dimension:\n", neg_index_2.size())
        neg_1 = score_mat[[range(config.BATCH_SIZE), neg_index_1]] # hard negatives for GEX 
        neg_2 = score_mat[[neg_index_2, range(config.BATCH_SIZE)]] # hard negatives for ATAC
        # print("neg_1:\n", neg_1); print("neg_2:\n", neg_2)
        
        loss_1 = torch.max(self.margin - true_score + neg_1, torch.zeros(1, config.BATCH_SIZE).to(config.DEVICE))
        loss_2 = torch.max(self.margin - true_score + neg_2, torch.zeros(1, config.BATCH_SIZE).to(config.DEVICE))

        return torch.mean(loss_1 + loss_2)

    def crossEntropy(self, score_mat):

        batch_size = score_mat.size()[0]
        target = torch.arange(batch_size)

        loss_1 = self.cross_entropy_loss(score_mat, target.to(config.DEVICE))
        loss_2 = self.cross_entropy_loss(score_mat.T, target.to(config.DEVICE))

        return 0.5 * (loss_1 + loss_2)

    def cellTypeMatchingProbRow(self, score_mat, cell_type):

        # Collect list of index list for each cell type
        idx_in_type = collections.defaultdict(list)
        for i, x in enumerate(cell_type):
            idx_in_type[x].append(i)

        # Compute matching probs for each cell type
        score_mat_norm = score_mat.softmax(dim = 0)
        probs = []
        for idx in idx_in_type.values():
            prob_type = 0
            for i in idx:
                prob_type += score_mat_norm[i, idx].sum()
            probs.append(prob_type / len(idx)) 

        # Take average of matching prob from cell types
        ct_match_prob = torch.tensor(probs).mean()

        return ct_match_prob

    def cellTypeMatchingProb(self, score_mat, cell_type):
        row = self.cellTypeMatchingProbRow(score_mat, cell_type) # Softmax on rows (normalize GEX)
        col = self.cellTypeMatchingProbRow(score_mat.T, cell_type) # Softmax on cols (normalize ATAC) 
        
        return 0.5 * (row + col)

    def forward(self, gex_out_0, gex_out_1, atac_out_0, atac_out_1, cell_type):
      
        score_mat = self.similarityScore(gex_out_0, gex_out_1, atac_out_0, atac_out_1)#; print("score_mat:\n", score_mat)

        loss_triplet = self.triplet(score_mat)
        loss_cross = self.crossEntropy(score_mat)
        loss = loss_triplet + loss_cross
        # print(score_mat); print(cell_type)
        ct_match_prob = self.cellTypeMatchingProb(score_mat, cell_type)

        return loss.to(config.DEVICE), loss_triplet, loss_cross, ct_match_prob

In [35]:
# sum_score_mat=torch.zeros(len(idx_in_type.values()),len(idx_in_type.values()))
#     for i, dx in enumerate(idx_in_type.values()):
#       for j, dx2 in enumerate(idx_in_type.values()):
#         tem=score_mat[np.ix_(dx, dx2)].sum()
#         print(tem)
#         sum_score_mat[i,j]=tem
#     print("score_mat before softmax")
#     print(sum_score_mat)
#     score_mat_norm = 0.5 * (sum_score_mat.softmax(dim = 0) + sum_score_mat.softmax(dim = 1))
#     print(sum_score_mat)
#     #return torch.tensor(probs).mean()
#     return np.sum(np.diagonal(score_mat_norm))

In [36]:
# TEST
# import random
# random.seed(0)
# gex_mat=torch.randn([5,64])
# # print(gex_mat[0,])
# atac_mat=torch.randn([5,64])
# # print(atac_mat[0,])
# # print(torch.dot(gex_mat[0,:32],atac_mat[0,:32]))
# loss=bidirectTripletLoss(alpha=0.2,margin=1)
# res=loss(gex_mat,atac_mat)
# res

In [37]:
# def cellTypeMatchingProbRow(score_mat, cell_type):

#     # Collect list of index list for each cell type
#     idx_in_type = collections.defaultdict(list)
#     for i, x in enumerate(cell_type):
#         idx_in_type[x].append(i)

#     # Compute matching probs for each cell type
#     score_mat_norm = score_mat.softmax(dim = 0)
#     print('score_mat_norm: \n', score_mat_norm)
#     probs = []
#     for idx in idx_in_type.values():
#         print('idx: ', idx)
#         prob_type = 0
#         for i in idx:
#             prob_type += score_mat_norm[i, idx].sum()
#         print('prob_type: ', prob_type)
#         probs.append(prob_type / len(idx)) 
#     print('probs: ', probs, "\n")

#     # Take average of matching prob from cell types
#     ct_match_prob = torch.tensor(probs).mean()

#     return ct_match_prob

# def cellTypeMatchingProb(score_mat, cell_type):
#     row = cellTypeMatchingProbRow(score_mat, cell_type) # Softmax on rows (normalize GEX)
#     col = cellTypeMatchingProbRow(score_mat.T, cell_type) # Softmax on cols (normalize ATAC) 
    
#     return 0.5 * (row + col)

# random.seed(1)
# mat=torch.randn([5,5])
# cell_type=['a','a','b','a','b']
# print(mat)
# print(cellTypeMatchingProb(mat, cell_type))

In [38]:
class MultiomeDataset(Dataset):
    def __init__(
        self, csr_gex, csr_atac, cell_type
    ):
        super().__init__()
        
        self.csr_gex = csr_gex
        self.csr_atac = csr_atac
    
    def __len__(self):
        return self.csr_gex.shape[0]
    
    def __getitem__(self, index: int):
        x_gex = torch.tensor(self.csr_gex[index,:].todense())
        x_atac = torch.tensor(self.csr_atac[index,:].todense())
        return {'gex':x_gex, 'atac':x_atac, 'cell_type':cell_type[index]}
  
def get_dataloaders(gex_train, atac_train, cell_type_train,  gex_val, atac_val, cell_type_val):
    
    # mod2_train = mod2_train.iloc[sol_train.values.argmax(1)]
    # mod2_test = mod2_test.iloc[sol_test.values.argmax(1)]
    
    dataset_train = MultiomeDataset(gex_train, atac_train, cell_type_train)
    data_train = DataLoader(dataset_train, config.BATCH_SIZE, shuffle = True, num_workers = config.NUM_WORKERS)
    
    dataset_val = MultiomeDataset(gex_val, atac_val, cell_type_val)
    data_val = DataLoader(dataset_val, config.BATCH_SIZE, shuffle = False, num_workers = config.NUM_WORKERS)
    
    return data_train, data_val

In [11]:
index = get_chr_index(ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/ATAC_processed.h5ad"))

In [12]:
batch = ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/GEX_processed.h5ad").obs['batch']
batch = list(batch)
train_id = [a for a, l in enumerate(batch) if l not in ['s2d4','s1d1']]
val_id =  [a for a, l in enumerate(batch) if l == 's1d1']
test_id = [a for a, l in enumerate(batch) if l == 's2d4']

cell_type = ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/GEX_processed.h5ad").obs['cell_type']

csr_gex = ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/GEX_processed.h5ad").layers['log_norm']
csr_atac = ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/ATAC_processed.h5ad").layers['log_norm']

In [39]:
import random
random.seed(0)

idx_train = [train_id[i] for i in random.sample(range(0, 45000), 10240)]
gex_train = csr_gex[idx_train,:]
atac_train = csr_atac[idx_train,:]
cell_type_train = [cell_type[j] for j in idx_train]

idx_val = [train_id[i] for i in random.sample(range(0, 6000), 64)]
gex_val = csr_gex[idx_val,:]
atac_val = csr_atac[idx_val,:]
cell_type_val = [cell_type[j] for j in idx_val]

data_train, data_val = get_dataloaders(gex_train, atac_train, cell_type_train, gex_val, atac_val, cell_type_val)

In [53]:
criterion = bidirectTripletLoss(alpha = 0.2, margin = 0.5).to(config.DEVICE)
model = Encoder(kernel_size_gex = 100, kernel_size_atac_1 = 50, kernel_size_atac_2 = 10, index = index).to(config.DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = config.LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 100, gamma = 0.1)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones = [100], gamma = 0.1)

In [52]:
def train(model, data_train, epochs):

  model.train()

  for e in range(epochs):
    running_loss = 0.0
    running_loss_triplet = 0.0
    running_ct_prob = 0.0
    for iter, data in enumerate(data_train):
      gex_input = data['gex'].to(config.DEVICE)
      atac_input = data['atac'].to(config.DEVICE)
      cell_type_input = data['cell_type']
      # print(cell_type_input)

      model.zero_grad()
      optimizer.zero_grad()

      ### Forward
      gex_out_0, gex_out_1, atac_out_0, atac_out_1 = model(gex_input, atac_input)

      ### Compute loss
      loss, loss_triplet, loss_cross, ct_match_prob = criterion(gex_out_0, gex_out_1, atac_out_0, atac_out_1, cell_type_input)
      
      ### Propagate loss
      # running_loss += loss.item()
      # loss.backward()
      running_loss += loss_cross.item()
      loss_cross.backward()

      running_ct_prob += ct_match_prob.item()
      running_loss_triplet += loss_triplet.item()

      ### update parameters
      optimizer.step()
    
    scheduler.step()
    
    print('triplet_loss = ', loss_triplet.item())
    print('cross_loss = ', loss_cross.item())
    if (e+1) % 10 == 0: 
      print('Epoch-{0}: lr = {1}, loss = {2}, triplet loss = {3}, cell type match prob = {4}'.format(
          e+1, 
          optimizer.param_groups[0]['lr'], 
          running_loss / len(data_train), 
          running_loss_triplet / len(data_train), 
          running_ct_prob / len(data_train)
          )
      )

In [ ]:
train(model, data_train, epochs = 100) # linear1: dim=30; linear2: dim = 300

triplet_loss =  1.1460438966751099
cross_loss =  3.360142469406128
triplet_loss =  1.0753529071807861
cross_loss =  3.305295467376709
triplet_loss =  1.0864968299865723
cross_loss =  3.30295467376709
triplet_loss =  1.1095473766326904
cross_loss =  3.2858095169067383
triplet_loss =  1.1107178926467896
cross_loss =  3.310652256011963
triplet_loss =  1.0497989654541016
cross_loss =  3.2400331497192383
triplet_loss =  1.0351252555847168
cross_loss =  3.2167253494262695
triplet_loss =  1.0159025192260742
cross_loss =  3.2340831756591797
triplet_loss =  0.9763484597206116
cross_loss =  3.196658134460449
triplet_loss =  1.0023410320281982
cross_loss =  3.220916271209717
Epoch-10: lr = 0.0005, loss = 3.2105545714497565, triplet loss = 0.9952025681734085, cell type match prob = 0.09227101658470929
triplet_loss =  0.9872111082077026
cross_loss =  3.202328681945801
triplet_loss =  0.9705946445465088
cross_loss =  3.2252254486083984
triplet_loss =  0.9720903038978577
cross_loss =  3.1847848892211

In [42]:
train(model, data_train, epochs = 100) # linear1: dim=10; linear2: dim = 300

triplet_loss =  1.2339603900909424
cross_loss =  3.3827321529388428
triplet_loss =  1.1104178428649902
cross_loss =  3.3224382400512695
triplet_loss =  1.1170684099197388
cross_loss =  3.3244125843048096
triplet_loss =  1.0749058723449707
cross_loss =  3.306784152984619
triplet_loss =  1.0510550737380981
cross_loss =  3.2733216285705566
triplet_loss =  1.0594245195388794
cross_loss =  3.2720608711242676
triplet_loss =  1.0505023002624512
cross_loss =  3.2911014556884766
triplet_loss =  1.0496585369110107
cross_loss =  3.28818941116333
triplet_loss =  1.0251787900924683
cross_loss =  3.2238216400146484
triplet_loss =  1.0529909133911133
cross_loss =  3.239223003387451
Epoch-10: lr = 0.0005, loss = 3.2357745498418806, triplet loss = 1.02143522426486, cell type match prob = 0.09219703325070441
triplet_loss =  1.0032559633255005
cross_loss =  3.2199857234954834
triplet_loss =  0.9937990307807922
cross_loss =  3.218421459197998
triplet_loss =  1.0321228504180908
cross_loss =  3.228871822357

In [43]:
train(model, data_train, epochs = 200)

triplet_loss =  0.8387348651885986
cross_loss =  3.1357779502868652
triplet_loss =  0.8659029006958008
cross_loss =  3.1367273330688477
triplet_loss =  0.8449511528015137
cross_loss =  3.1388092041015625
triplet_loss =  0.8470821380615234
cross_loss =  3.135875940322876
triplet_loss =  0.8762235045433044
cross_loss =  3.134636878967285
triplet_loss =  0.8199192881584167
cross_loss =  3.138876438140869
triplet_loss =  0.8215453624725342
cross_loss =  3.137922763824463
triplet_loss =  0.8848443031311035
cross_loss =  3.151812791824341
triplet_loss =  0.8109655380249023
cross_loss =  3.1280181407928467
triplet_loss =  0.8421362638473511
cross_loss =  3.1372580528259277
Epoch-10: lr = 5e-05, loss = 3.135118205845356, triplet loss = 0.8444276101887226, cell type match prob = 0.09556700801476836
triplet_loss =  0.8626747131347656
cross_loss =  3.1307477951049805
triplet_loss =  0.8356744050979614
cross_loss =  3.141845703125
triplet_loss =  0.8313072323799133
cross_loss =  3.129523515701294


KeyboardInterrupt: ignored

In [ ]:
# def train(model, data_train, epochs):

#   model.train()

#   for e in range(epochs):
#     running_loss = 0.0
#     for iter, data in enumerate(data_train):
#       gex_input = data['gex'].to(config.DEVICE)
#       atac_input = data['atac'].to(config.DEVICE)

#       model.zero_grad()
#       optimizer.zero_grad()

#       # forward
#       gex_out_0, gex_out_1, atac_out_0, atac_out_1 = model(gex_input, atac_input)
#       # print(gex_out_0); print(gex_out_1); print(atac_out_0); print(atac_out_1); 

#       loss, loss_triplet, loss_cross = criterion(gex_out_0, gex_out_1, atac_out_0, atac_out_1)
      
#       # running_loss += loss.item()
#       # loss.backward()
#       running_loss += loss_triplet.item()
#       loss_triplet.backward()

#       # update parameters
#       optimizer.step()
    
#     # scheduler.step()
    
#     print('triplet_loss = ', loss_triplet); print('cross_loss = ', loss_cross)
#     if (e+1) % 10 == 0: 
#       # print('Epoch-{0} lr: {1}'.format(e+1, optimizer.param_groups[0]['lr']))
#       print('epoch[%d] = %.8f' % (e+1, running_loss / len(data_train)))


# train(model, data_train, epochs = 100)

In [ ]:
# train(model, data_train, epochs = 100)

In [ ]:
torch.save(model.state_dict(), 'drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/model/trained_model_10240cells')

In [ ]:
def inference(model, data_val):

    # Initialize encoder & decoder 
    model.eval()
    model.to(config.DEVICE)
    criterion.to(config.DEVICE)
    
    for iter, data in enumerate(data_val):
      gex_input = data['gex'].to(config.DEVICE)
      atac_input = data['atac'].to(config.DEVICE)
      cell_type_input = data['cell_type']

      print(collections.Counter(cell_type_input))

      ### Forward
      gex_out_0, gex_out_1, atac_out_0, atac_out_1 = model(gex_input, atac_input)

      ### Compute loss
      loss, loss_triplet, loss_cross, ct_match_prob = criterion(gex_out_0, gex_out_1, atac_out_0, atac_out_1, cell_type_input)

    return loss, loss_triplet, loss_cross, ct_match_prob



In [ ]:
model = Encoder(kernel_size_gex = 100, kernel_size_atac_1 = 50, kernel_size_atac_2 = 10, index = index).to(config.DEVICE)
# criterion = bidirectTripletLoss(alpha = 0.2, margin = 0.5).to(config.DEVICE)
model.load_state_dict(torch.load('drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/model/trained_model_10240cells'))
loss, loss_triplet, loss_cross, ct_match_prob = inference(model, data_val)

Counter({'CD14+ Mono': 12, 'Erythroblast': 8, 'CD8+ T': 7, 'CD4+ T activated': 7, 'Normoblast': 7, 'CD4+ T naive': 6, 'NK': 5, 'B1 B': 4, 'Naive CD20+ B': 3, 'CD16+ Mono': 2, 'Lymph prog': 1, 'Transitional B': 1, 'ILC': 1})


In [ ]:
print('loss = {0}, triplet loss = {1}, cell type match prob = {2}'.format(loss, loss_triplet, ct_match_prob))

loss = 5.09748649597168, triplet loss = 1.533463716506958, cell type match prob = 0.0909629762172699
